# Tweepyでfollowerの共通部分を取得
## API.friends_ids(id/screen_name/user_id[, cursor]) return list of Integers
- **id**がフォローしている人のidを返す
- [Doc](https://kurozumi.github.io/tweepy/api.html?highlight=cursor#API.friends_ids)
- **id** – Specifies the ID or screen name of the user.
- **screen_name** – Specifies the screen name of the user. Helpful for disambiguating when a valid screen name is also a user ID.
- **user_id** – Specifies the ID of the user. Helpful for disambiguating when a valid user ID is also a valid screen name.
- **cursor** – Breaks the results into pages. Provide a value of -1 to begin paging. Provide values as returned to in the response body’s next_cursor and previous_cursor attributes to page back and forth in the list.

## API.followers_ids(id/screen_name/user_id) return list of Integers
- **id**をフォローしている人のidを返す
- [Doc](https://kurozumi.github.io/tweepy/api.html?highlight=cursor#API.followers_ids)
- **id** – Specifies the ID or screen name of the user.
- **screen_name** – Specifies the screen name of the user. Helpful for disambiguating when a valid screen name is also a user ID.
- **user_id** – Specifies the ID of the user. Helpful for disambiguating when a valid user ID is also a valid screen name.
- **cursor** – Breaks the results into pages. Provide a value of -1 to begin paging. Provide values as returned to in the response body’s next_cursor and previous_cursor attributes to page back and forth in the list.


## lookup_users
- 一度に100件までユーザー情報を取得できる
- リファレンスにはないが、ソースコードには存在↓
- [Doc](https://github.com/tweepy/tweepy/blob/fdf133184a918a64a0853dc52bffddc90570dcfa/tweepy/api.py#L310-L321)
```python
def lookup_users(self, user_ids=None, screen_names=None, include_entities=None):
        """ Perform bulk look up of users from user ID or screenname """
        post_data = {}
        if include_entities is not None:
            include_entities = 'true' if include_entities else 'false'
            post_data['include_entities'] = include_entities
        if user_ids:
            post_data['user_id'] = list_to_csv(user_ids)
        if screen_names:
            post_data['screen_name'] = list_to_csv(screen_names)

        return self._lookup_users(post_data=post_data)
```

In [0]:
import tweepy
import pandas as pd

In [0]:
def my_auth (consumer_key, consumer_secret, access_token, access_token_secret):
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)
  return tweepy.API(auth)

def friends_common_part(api, *args):
  candidate_dict = {}
  if len(args) < 2:
    return "error"
  
  common_set = set(api.friends_ids(args[0]))
  for i in range(1, len(args)):
    common_set = common_set & set(api.friends_ids(args[i]))
  
  common_list = list(common_set)
  
  for i in range(0, len(common_list), 100):
    for user in api.lookup_users(user_ids=common_list[i:i+100]):
      candidate_dict[user.id] = {}
      candidate_dict[user.id]['name'] = user.name
      candidate_dict[user.id]['twitter_page'] = 'https://twitter.com/' + user.screen_name
      candidate_dict[user.id]['my_page'] = user.url
      candidate_dict[user.id]['description'] = user.description
  return candidate_dict

def followers_common_part(api, *args):
  candidate_dict = {}
  if len(args) < 2:
    return "error"
  
  common_set = set(api.friends_ids(args[0]))
  for i in range(1, len(args)):
    common_set = common_set & set(api.followers_ids(args[i]))
  
  common_list = list(common_set)
  
  for i in range(0, len(common_list), 100):
    for user in api.lookup_users(user_ids=common_list[i:i+100]):
      candidate_dict[user.id] = {}
      candidate_dict[user.id]['name'] = user.name
      candidate_dict[user.id]['twitter_page'] = 'https://twitter.com/' + user.screen_name
      candidate_dict[user.id]['my_page'] = user.url
      candidate_dict[user.id]['description'] = user.description
  return candidate_dict

↓ あまり実行しすぎると 15分の通信制限がかかるので注意
（APIを叩く処理のみ）

In [0]:
# Twitter Developer Account Credentials
consumer_key = 'hogehoge'
consumer_secret = 'hogehoge'
access_token = 'hogehoge'
access_token_secret = 'hogehoge'
  
# 影響力のある人のID（複数人可能）
user_ids = [
    'genya0407',
    'EbiEbiEvidence',
    'alohat_jp'
]

#上アカウントがフォローしている人の共通部分
candidate_friends = friends_common_part(
    my_auth(consumer_key, consumer_secret, access_token, access_token_secret),
    *user_ids
)

↓表の表示（API関係なし）

In [0]:
candidate_friends_df = pd.DataFrame(candidate_friends)
candidate_friends_df.T[['name','twitter_page','description','my_page']]

,name,twitter_page,description,my_page
1113768585577488384,Shota Hashimoto,https://twitter.com/shotaH_tw,見習いengineer/figure skate お気軽にフォローしてくださいませ〜,None
1055057801951596544,tatatatataro,https://twitter.com/taro_jxv_happy,🎾🏂🍻🥟🇳🇵☀️,https://t.co/4u3d2irH9h
1112714694588928000,社会的ミーくん,https://twitter.com/knock_9999,三茶の殺し屋 500yen/man,None
1051728214203277312,渡邉 洸 / Ko Watanabe,https://twitter.com/ko_watanabe_tw,渋谷でエンジニア / NAISTユビ研卒 / トビタテ8期🇩🇪 / 年間100冊計画実施中(...,https://t.co/kuwJU7SAF9
862019745545035776,なゆたいむ,https://twitter.com/nayutime,ｿﾌﾄｳｪｱｴﾝｼﾞﾆｱ。分散ｺﾝﾋﾟｭｰﾃｨﾝｸﾞのﾌﾟﾛということになってます。,None
86075525,そすうぽよ,https://twitter.com/_primenumber,会社員になった | プログラミング | AtCoder黄 | オセロAI | 私が真理です ...,https://t.co/YV7kdAtNPx
1113818641336688641,NakaS,https://twitter.com/StartedTwittter,何かになりたい渋谷のエンジニア,None
56052608,少し考えて行動する,https://twitter.com/ioriveur,こうした過酷な土地では死も生活の一部ではあるが、一つ一つの死が非常に鋭い痛みとして感じられる,https://t.co/kTvnbPXHKB
1112926435977523201,はっしー,https://twitter.com/kure4_Mars,尋常じゃないくらいゲームしてて気づいたらゲームエンジニアになってたです,None
1359669258,Liva@ドイツ（衣食住満ち足りて礼節を勉強中）,https://twitter.com/liva_jy,普段はご飯のツイートしかしません / PFLab(ktu研) / IS15er / '16未...,https://t.co/CVxneKU6Dn


In [0]:
#上アカウントをフォローしている人の共通部分
candidate_followers = followers_common_part(
    my_auth(consumer_key, consumer_secret, access_token, access_token_secret),
    *user_ids
)

In [0]:
candidate_followers_df = pd.DataFrame(candidate_followers)
candidate_followers_df.T[['name','twitter_page','description','my_page']]

,name,twitter_page,description,my_page
1113768585577488384,Shota Hashimoto,https://twitter.com/shotaH_tw,見習いengineer/figure skate お気軽にフォローしてくださいませ〜,None
1055057801951596544,tatatatataro,https://twitter.com/taro_jxv_happy,🎾🏂🍻🥟🇳🇵☀️,https://t.co/4u3d2irH9h
1112714694588928000,社会的ミーくん,https://twitter.com/knock_9999,三茶の殺し屋 500yen/man,None
1113818641336688641,NakaS,https://twitter.com/StartedTwittter,何かになりたい渋谷のエンジニア,None
1051728214203277312,渡邉 洸 / Ko Watanabe,https://twitter.com/ko_watanabe_tw,渋谷でエンジニア / NAISTユビ研卒 / トビタテ8期🇩🇪 / 年間100冊計画実施中(...,https://t.co/kuwJU7SAF9
86075525,そすうぽよ,https://twitter.com/_primenumber,会社員になった | プログラミング | AtCoder黄 | オセロAI | 私が真理です ...,https://t.co/YV7kdAtNPx
862019745545035776,なゆたいむ,https://twitter.com/nayutime,ｿﾌﾄｳｪｱｴﾝｼﾞﾆｱ。分散ｺﾝﾋﾟｭｰﾃｨﾝｸﾞのﾌﾟﾛということになってます。,None
56052608,少し考えて行動する,https://twitter.com/ioriveur,こうした過酷な土地では死も生活の一部ではあるが、一つ一つの死が非常に鋭い痛みとして感じられる,https://t.co/kTvnbPXHKB
1112926435977523201,はっしー,https://twitter.com/kure4_Mars,尋常じゃないくらいゲームしてて気づいたらゲームエンジニアになってたです,None
1359669258,Liva@ドイツ（衣食住満ち足りて礼節を勉強中）,https://twitter.com/liva_jy,普段はご飯のツイートしかしません / PFLab(ktu研) / IS15er / '16未...,https://t.co/CVxneKU6Dn


↓ CSV書き出し

In [0]:
#candidate_friends_df.to_csv("candidate_friends.csv")
#candidate_followers_df.to_csv("candidate_followers.csv")